## 2 持续优化 LlamaIndex

### 1 从提示词与模型角度入手

- 通过提示词让大模型反思、总结，补充用户对话的信息不足
- 通过大模型与embeddings模型的选择，让大模型更符合业务需求

### 2 从业务角度入手

- 文本分割方法调整
- 如何处理新增内容
- 如何更改向量数据库



#### 2.1 理论知识：怎么定义分段(chunk)，怎么直接查看分段

##### 文本分块参数说明

大模型的单位 Token，100个 Token 约等于 75个汉字

##### chunk_size（分块大小）
- 定义了每个文本块的最大长度（通常以字符或token为单位）
- 较大的chunk_size意味着：
  - 每个块包含更多上下文信息
  - 可能会提高答案的准确性
  - 但会增加处理时间和内存使用
- 较小的chunk_size意味着：
  - 处理更快，内存使用更少
  - 但可能会丢失一些上下文信息

##### chunk_overlap（分块重叠）
- 定义了相邻文本块之间重叠的部分大小
- 重叠的目的是：
  - 保持文本块之间的连贯性
  - 避免在分块边界处丢失重要信息
  - 确保跨越多个块的概念能被完整捕获

##### 建议值
##### 对于中文文本：
- **chunk_size**: 通常设置在 200-500 之间
- **chunk_overlap**: 通常设置为 chunk_size 的 10%-20%

> 注：具体值需要根据您的应用场景和文本特点来调整


Web Page Reader

https://docs.llamaindex.ai/en/stable/examples/data_connectors/WebPageDemo/#using-trafilaturawebreader

%pip install llama-index-readers-web trafilatura

In [40]:
from llama_index.readers.web import TrafilaturaWebReader
 
documents = TrafilaturaWebReader().load_data(
         ["https://baike.baidu.com/item/Transformer模型架构",
          "https://baike.baidu.com/item/LangChain"]
)

print(f"文档数量是 {len(documents)} 个")
print("第一个文档的前200个字符：")
print(documents[0].text[:200])
print(len(documents[0].text))       



from llama_index.core.node_parser import SimpleNodeParser


# 解析器(parser)用于分割文档(document)为节点(node)
node_parser = SimpleNodeParser().from_defaults(
    chunk_size=40,
    chunk_overlap=10
)

print(f"Node解析器：{node_parser}")

# 分割文档
nodes = node_parser.get_nodes_from_documents(documents)

print(f"分割后的节点数量：{len(nodes)}")
print(f"第一个节点的内容：{nodes[0].text}")
print(f"第一个节点的长度：{len(nodes[0].text)}")
print(f"第二个节点的内容：{nodes[1].text}")
print(f"第二个节点的长度：{len(nodes[1].text)}")
print(f"第三个节点的内容：{nodes[2].text}")
print(f"第三个节点的长度：{len(nodes[2].text)}")

print("--------------------------------")
print(f"完整的Document：\n    {documents[0]}")
print(f"完整的Node1：\n    {nodes[0]}")
print(f"完整的Node2：\n    {nodes[1]}")
print(f"完整的Node3：\n    {nodes[2]}")
print("--------------------------------")
print(f"Node关系1：\n    {nodes[0].relationships}")
print(f"Node关系2：\n    {nodes[1].relationships}")
print(f"Node关系3：\n    {nodes[2].relationships}")


文档数量是 2 个
第一个文档的前200个字符：
Transformer模型架构，是2017 年，Google 在论文 Attentions is All you need 中提出的模型，其使用 Self-Attention 结构取代了在 NLP 任务中常用的 RNN 网络结构。相比 RNN 网络结构，其最大的优点是可以并行计算。
Transformer 本质上是一个 Encoder-Decoder 架构。因此中间部分的 Transformer 
1133
Node解析器：include_metadata=True include_prev_next_rel=True callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001AE13E0DC90> id_func=<function default_id_func at 0x000001AE10A34A40> chunk_size=40 chunk_overlap=10 separator=' ' paragraph_separator='\n\n\n' secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'
Metadata length (0) is close to chunk size (40). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (0) is close to chunk size (40). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
分割后的节点数量：24
第一个节点的内容：Transformer模型架构，是2017 年，Google 在论文 Attentions is All yo

#### 2.1 文本分割方法调整-- JSON 解析器

In [ ]:
def create_unstructured_db(db_name:str,label_name:list):
    print(f"知识库名称为：{db_name}，类目名称为：{label_name}")
    if label_name is None:
        gr.Info("没有选择类目")
    elif len(db_name) == 0:
        gr.Info("没有命名知识库")
    # 判断是否存在同名向量数据库
    elif db_name in os.listdir(DB_PATH):
        gr.Info("知识库已存在，请换个名字或删除原来知识库再创建")
    else:
        gr.Info("正在创建知识库，请等待知识库创建成功信息显示后前往RAG问答")
        nodes = []
        for label in label_name:
            label_path = os.path.join(UNSTRUCTURED_FILE_PATH,label)
            nodes.extend(SimpleDirectoryReader(label_path).load_data())

        # 添加json解析和句子窗口解析
        from llama_index.core.node_parser import JSONNodeParser, SentenceWindowNodeParser
        
        # 先用JSON解析器处理
        json_parser = JSONNodeParser()
        nodes = json_parser.get_nodes_from_documents(nodes, show_progress=True)
        
        # 创建索引并保存
        index = VectorStoreIndex(nodes)
        db_path = os.path.join(DB_PATH, db_name)
        if not os.path.exists(db_path):
            os.mkdir(db_path)
        index.storage_context.persist(db_path)
        gr.Info("知识库创建成功，可前往RAG问答进行提问")


#### 2.1 文本分割方法调整-- 句子窗口解析器
[ Llamaindex 官方文档 ：基本 RAG 流程](https://docs.llamaindex.ai/en/stable/understanding/rag/)


## 基本 RAG 的工作流程与局限性

### 基本 RAG 流程
1. **文档分块**：按指定的 chunk size 将文档切分
2. **向量化处理**：对文档块进行 embedding 向量化
3. **相似度检索**：计算用户问题与文档块的相似度，选取 Top-K 相关文档作为 context
4. **LLM 生成**：将 context 和用户问题一起发送给 LLM 生成回答

### 核心问题：chunk size 的两难困境

### 小 chunk size：
- ✅ 与问题的匹配度高
- ❌ context 信息量不足
- ❌ 可能导致回答质量下降

### 大 chunk size：
- ✅ context 信息量充足
- ❌ 与问题的匹配精度下降
- ❌ 同样可能影响回答质量

### 句子窗口解析器(Sentence Window Parser)的优势

### 核心思想
在保持高匹配精度的同时，通过上下文窗口提供更多相关信息

### 工作方式
1. 先将文档切分为较小的文档块
2. 匹配到相关文档块后，自动获取其周围的文档内容作为补充 context
3. 通过 `window_size` 参数控制获取周围内容的范围

### 主要参数
```python
SentenceWindowNodeParser.from_defaults(
    window_size=2,                            # 在匹配块两侧各取2个句子
    window_metadata_key="window",             # 窗口信息的元数据键
    original_text_metadata_key="original_text" # 原始文本的元数据键
)
```

这种方法既保证了检索的精确性，又能提供足够的上下文信息，有效解决了基本 RAG 中 chunk size 的两难问题。

In [54]:
from llama_index.core.node_parser import SentenceWindowNodeParser
 
#定义句子窗口 Node 解析器
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
 
print(f"Node解析器：{node_parser}")
print("--------------------------------")

from llama_index.core import Document
 
text = "公安机关调取了一份行政机关收集的视听资料作为证据，但该视听资料已经被改动过。这份视听资料是否可以作为证据使用呢？"


#  处理中文标点，否则不会分割
# text = text.replace("。", ". ") \
#          .replace("？", "? ") \
#          .replace("！", "! ") \
#          .replace("；", "; ") \
#          .replace("：", ": ") \
#          .replace("，", ", ") \
#          .replace("、", ", ")

nodes = node_parser.get_nodes_from_documents([Document(text=text)])

print([x.text for x in nodes])
print("窗口数据和文档数据")
print(nodes[0].metadata)


Node解析器：include_metadata=True include_prev_next_rel=True callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000001AE16D2C810> id_func=<function default_id_func at 0x000001AE10A34A40> sentence_splitter=<function split_by_sentence_tokenizer.<locals>.<lambda> at 0x000001AE166C1440> window_size=3 window_metadata_key='window' original_text_metadata_key='original_text'
--------------------------------
['公安机关调取了一份行政机关收集的视听资料作为证据，但该视听资料已经被改动过。这份视听资料是否可以作为证据使用呢？']
窗口数据和文档数据
{'window': '公安机关调取了一份行政机关收集的视听资料作为证据，但该视听资料已经被改动过。这份视听资料是否可以作为证据使用呢？', 'original_text': '公安机关调取了一份行政机关收集的视听资料作为证据，但该视听资料已经被改动过。这份视听资料是否可以作为证据使用呢？'}


In [ ]:
# 创建非结构化向量数据库
def create_unstructured_db(db_name:str,label_name:list):
    print(f"知识库名称为：{db_name}，类目名称为：{label_name}")
    if label_name is None:
        gr.Info("没有选择类目")
    elif len(db_name) == 0:
        gr.Info("没有命名知识库")
    # 判断是否存在同名向量数据库
    elif db_name in os.listdir(DB_PATH):
        gr.Info("知识库已存在，请换个名字或删除原来知识库再创建")
    else:
        gr.Info("正在创建知识库，请等待知识库创建成功信息显示后前往RAG问答")
        nodes = []
        for label in label_name:
            label_path = os.path.join(UNSTRUCTURED_FILE_PATH,label)
            nodes.extend(SimpleDirectoryReader(label_path).load_data())

        # 添加json解析和句子窗口解析
        from llama_index.core.node_parser import JSONNodeParser, SentenceWindowNodeParser
        
        # 创建一个新的列表存储解析后的节点
        parsed_nodes = []
        
        # 根据文档类型选择合适的解析器
        for doc in nodes:
            if doc.metadata.get("file_type") == "json":
                # 如果是JSON文档，使用JSON解析器
                json_parser = JSONNodeParser()
                doc_parsed_nodes = json_parser.get_nodes_from_documents([doc], show_progress=True)
            else:
                # 如果是普通文本，直接使用句子窗口解析器
                sentence_parser = SentenceWindowNodeParser.from_defaults(
                    window_size=2,
                    window_metadata_key="window",
                    original_text_metadata_key="original_text"
                )
                doc_parsed_nodes = sentence_parser.get_nodes_from_documents([doc], show_progress=True)
            parsed_nodes.extend(doc_parsed_nodes)
        
        # 将解析后的节点添加到原始节点列表
        nodes.extend(parsed_nodes)
    
        # 创建索引并保存
        index = VectorStoreIndex(nodes)
        db_path = os.path.join(DB_PATH, db_name)
        if not os.path.exists(db_path):
            os.mkdir(db_path)
        index.storage_context.persist(db_path)
        gr.Info("知识库创建成功，可前往RAG问答进行提问")

#### 2.1 文本分割方法调整-- 组合解析器

In [ ]:
from llama_index.core.node_parser import ComposableNodeParser

# 创建组合解析器
composable_parser = ComposableNodeParser.from_defaults(
    nodes_parsers=[
        JSONNodeParser(),
        SentenceWindowNodeParser.from_defaults(
            window_size=3,
            window_metadata_key="window",
            original_text_metadata_key="original_text"
        )
    ]
)

# 使用组合解析器
nodes = composable_parser.get_nodes_from_documents(documents)

#### 2.2 如何更新 Index 处理新增内容

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
def add_to_existing_db(db_name: str, new_files: list):
    """向现有知识库添加新文档，无需重建整个索引"""
    
    # 核心原理1: 检查并加载现有索引
    # - 向量数据库的持久化存储包含了索引结构和向量数据
    # - StorageContext 可以从持久化目录中恢复完整的索引状态
    if db_name not in os.listdir(DB_PATH):
        gr.Info("指定的知识库不存在")
        return
    
    db_path = os.path.join(DB_PATH, db_name)
    storage_context = StorageContext.from_defaults(persist_dir=db_path)
    index = load_index_from_storage(storage_context)
    
    # 核心原理2: 新文档的处理流程
    # - 使用 SimpleDirectoryReader 读取新文件
    # - 保持与原始索引创建时相同的文档处理方式
    new_documents = SimpleDirectoryReader(input_files=new_files).load_data()
    new_nodes = []
    
    # 核心原理3: 文档解析策略
    # - 保持与原始索引相同的解析方式，确保新旧节点的一致性
    # - JSON文档和普通文本使用不同的解析器
    for doc in new_documents:
        if doc.metadata.get("file_type") == "json":
            json_parser = JSONNodeParser()
            doc_parsed_nodes = json_parser.get_nodes_from_documents([doc], show_progress=True)
        else:
            # 使用句子窗口解析器处理普通文本
            # - window_size=2 表示在匹配句子前后各保留2个句子作为上下文
            sentence_parser = SentenceWindowNodeParser.from_defaults(
                window_size=2,
                window_metadata_key="window",
                original_text_metadata_key="original_text"
            )
            doc_parsed_nodes = sentence_parser.get_nodes_from_documents([doc], show_progress=True)
        new_nodes.extend(doc_parsed_nodes)
    
    # 核心原理4: 增量更新机制
    # - insert_nodes 方法实现了向量数据库的增量更新
    # - 只计算和存储新节点的向量，不影响现有的向量数据
    # - 新节点会被添加到现有的向量空间中，保持索引结构的完整性
    index.insert_nodes(new_nodes)
    
    # 核心原理5: 持久化更新
    # - persist 方法将更新后的索引状态保存到磁盘
    # - 只保存增量更新的部分，不会重写整个索引文件
    index.storage_context.persist(db_path)
    gr.Info("新文档已成功添加到知识库中")

# 调用方法： add_to_existing_db("知识库名称", ["新文件路径1", "新文件路径2"])

添加了新的 add_to_existing_db 函数，接受知识库名称和新文件列表作为参数
函数会：
检查知识库是否存在
加载现有的索引
处理新文件并创建节点
将新节点添加到现有索引中
保存更新后的索引

#### 2.3 更改向量数据库
在LlamaIndex中，默认使用的是SimpleVectorStore作为向量数据库，它是一个基于内存的简单向量存储实现。我们可以轻松地更改为其他向量数据库。以下是一些常用的替代方案：

1. ChromaDB：ChromaDB是一个开源的向量数据库，支持高效的向量搜索和存储。
2. Zilliz：Zilliz是一个分布式向量数据库，支持大规模数据存储和高效的向量搜索。
3. Faiss：Faiss是一个高效的向量搜索库，支持快速的向量索引和搜索。
4. Milvus：Milvus是一个分布式向量数据库，支持高效的向量搜索和存储。



In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from chromadb import PersistentClient
import chromadb

# ... existing code ...

def create_unstructured_db(db_name:str,label_name:list):
    # ... existing code ...
    
    # 创建Chroma客户端和集合
    chroma_client = PersistentClient(path=os.path.join(DB_PATH, db_name))
    chroma_collection = chroma_client.create_collection(name=db_name)
    
    # 创建向量存储
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # 创建索引
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    index.storage_context.persist()
    
    gr.Info("知识库创建成功，可前往RAG问答进行提问")

In [ ]:
# 使用Milvus向量数据库的示例
from llama_index.vector_stores.milvus import MilvusVectorStore

# ... existing code ...

def create_unstructured_db(db_name:str,label_name:list):
    # ... existing code ...
    
    # 创建Milvus向量存储
    vector_store = MilvusVectorStore(
        collection_name=db_name,
        dim=1536,  # 根据您使用的嵌入模型维度设置
        host="localhost",
        port=19530
    )
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # 创建索引
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    index.storage_context.persist()

In [ ]:
# 使用FAISS向量数据库的示例
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# ... existing code ...

def create_unstructured_db(db_name:str,label_name:list):
    # ... existing code ...
    
    # 创建FAISS向量存储
    dimension = 1536  # 根据您使用的嵌入模型维度设置
    faiss_index = faiss.IndexFlatL2(dimension)
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # 创建索引
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    index.storage_context.persist(os.path.join(DB_PATH, db_name))


主要步骤说明：

首先需要安装相应的向量数据库包，如：pip install chromadb、pip install pymilvus或pip install faiss-cpu
导入相应的向量存储类

创建向量存储实例

使用StorageContext配置存储上下文

在创建索引时使用自定义的存储上下文

每种向量数据库都有其特点：

Chroma: 轻量级、易于使用，支持持久化存储

Milvus: 分布式向量数据库，适合大规模生产环境

FAISS: Facebook开发的高性能向量检索库，适合大规模相似性搜索

选择哪种向量数据库主要取决于您的具体需求，如数据规模、查询性能要求、部署环境等。
